In [62]:
# Import Modules
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import datetime
from pandas_datareader import data as pdr
import yfinance as yf
from pandas.tseries.offsets import DateOffset


In [63]:
# use QQQ as the ticker
ticker = yf.Ticker("QQQ")

In [64]:
# Dataset period
ticker_df = ticker.history(start="2012-01-01", end="2021-12-31")


display(ticker_df.head())
display(ticker_df.tail())

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-01-03,51.671302,51.925526,51.526030,51.662224,39514100,0.0,0
2012-01-04,51.580514,51.952772,51.353527,51.880135,29403400,0.0,0
2012-01-05,51.771173,52.352259,51.571423,52.306862,41260600,0.0,0
2012-01-06,52.315944,52.606487,52.116198,52.488457,46325200,0.0,0
2012-01-09,52.624639,52.642799,52.143428,52.315937,39195500,0.0,0


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-12-23,394.309998,398.429993,394.140015,396.920013,29595500,0.0,0
2021-12-27,398.250000,403.480011,398.220001,403.480011,32820700,0.0,0
2021-12-28,404.399994,404.579987,400.709991,401.609985,33113700,0.0,0
2021-12-29,401.750000,403.049988,399.109985,401.549988,32248500,0.0,0
2021-12-30,401.329987,403.570007,399.750000,400.350006,23850100,0.0,0


In [65]:
# Filter the date index and close columns
signals_df = ticker_df.loc[:,["Close"]]

# Review the DataFrame
signals_df.tail()

,Close
Date,
2021-12-23,396.920013
2021-12-27,403.480011
2021-12-28,401.609985
2021-12-29,401.549988
2021-12-30,400.350006


In [66]:
# Use hvplot to visualize close trend
signals_df.hvplot(title='QQQ Close Price over 10 year period')

:Curve   [Date]   (Close)

In [67]:
# Set the variables for short window and long window periods
short_window = 7
long_window = 25

In [68]:
# Generate the short and long window simple moving averages
signals_df['SMA_fast'] = signals_df['Close'].rolling(window=short_window).mean()
signals_df['SMA_slow'] = signals_df['Close'].rolling(window=long_window).mean()

# Display the df
display(signals_df.head())
display(signals_df.tail())

,Close,SMA_fast,SMA_slow
Date,,,
2012-01-03,51.662224,NaN,NaN
2012-01-04,51.880135,NaN,NaN
2012-01-05,52.306862,NaN,NaN
2012-01-06,52.488457,NaN,NaN
2012-01-09,52.315937,NaN,NaN


,Close,SMA_fast,SMA_slow
Date,,,
2021-12-23,396.920013,389.725586,392.840396
2021-12-27,403.480011,390.716513,392.930095
2021-12-28,401.609985,392.897003,392.855508
2021-12-29,401.549988,395.344286,392.965881
2021-12-30,400.350006,398.152858,393.100962


In [69]:
# Create a column to hold the trading signal
signals_df['Signal'] = 0.0

In [70]:
# Generate the trading signal 0 or 1,
# where 1 for when the short sma crossess above the long sma
# set to 0 when no crossover

signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA_fast'][short_window:] > signals_df['SMA_slow'][short_window:], 1.0, 0.0
)

# Review the DataFrame
signals_df


,Close,SMA_fast,SMA_slow,Signal
Date,,,,
2012-01-03,51.662224,NaN,NaN,0.0
2012-01-04,51.880135,NaN,NaN,0.0
2012-01-05,52.306862,NaN,NaN,0.0
2012-01-06,52.488457,NaN,NaN,0.0
2012-01-09,52.315937,NaN,NaN,0.0
...,...,...,...,...
2021-12-23,396.920013,389.725586,392.840396,0.0
2021-12-27,403.480011,390.716513,392.930095,0.0
2021-12-28,401.609985,392.897003,392.855508,1.0


In [71]:
# Calculate when signal changes
# Identify trade entry (1) and exit (-1) points
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

# Review the DataFrame
signals_df

,Close,SMA_fast,SMA_slow,Signal,Entry/Exit
Date,,,,,
2012-01-03,51.662224,NaN,NaN,0.0,NaN
2012-01-04,51.880135,NaN,NaN,0.0,0.0
2012-01-05,52.306862,NaN,NaN,0.0,0.0
2012-01-06,52.488457,NaN,NaN,0.0,0.0
2012-01-09,52.315937,NaN,NaN,0.0,0.0
...,...,...,...,...,...
2021-12-23,396.920013,389.725586,392.840396,0.0,0.0
2021-12-27,403.480011,390.716513,392.930095,0.0,0.0
2021-12-28,401.609985,392.897003,392.855508,1.0,1.0


In [72]:
# Plot the exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='orange',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
exit



:Scatter   [Date]   (Close)

In [73]:
# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
entry

:Scatter   [Date]   (Close)

In [74]:
# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='black',
    ylabel='Price in $',
    width=1000,
    height=400)

# Show the plot
security_close


:Curve   [Date]   (Close)

In [75]:
# Visualize moving averages
moving_avgs = signals_df[['SMA_slow', 'SMA_fast']].hvplot(
    ylabel='Price',
    width=1000,
    height=400)

# Show the plot
moving_avgs

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

In [76]:
# Create the overlay plot
entry_exit_plot = security_close * moving_avgs * entry * exit

# Show the plot
entry_exit_plot.opts(
    title="QQQ - SMA Fast, SMA Slow, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)